<a href="https://colab.research.google.com/github/caiobarrosv/object_detection_for_grasping/blob/master/training_script_k_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!ls
!ls drive/My\ Drive/UFBA/Doutorado/Ssd_test

drive  sample_data
checkpoints  Dataset


# Instalações

In [9]:
!pip install neptune-client
!pip install mxnet-cu101
#==1.5.0
!pip install gluoncv
#==0.7.0

# Importações

In [10]:
# import argparse

# disable autotune


import os
import time
import numpy as np
import mxnet as mx
from mxnet import nd
from mxnet import gluon
from mxnet import autograd
import gluoncv as gcv
from gluoncv import data as gdata
from gluoncv import utils as gutils
from gluoncv.model_zoo import get_model
from gluoncv.data.batchify import Tuple, Stack, Pad
##<FASTER
from gluoncv.data.batchify import FasterRCNNTrainBatchify, Append # ,Tuple 
from gluoncv.data.transforms.presets.rcnn import FasterRCNNDefaultTrainTransform, \
    FasterRCNNDefaultValTransform
from gluoncv.utils.parallel import Parallel
from gluoncv.utils.metrics.rcnn import RPNAccMetric, RPNL1LossMetric, RCNNAccMetric, \
    RCNNL1LossMetric
from gluoncv.model_zoo.rcnn.faster_rcnn.data_parallel import ForwardBackwardTask
##/>
from gluoncv.data.transforms.presets.yolo import YOLO3DefaultTrainTransform
from gluoncv.data.transforms.presets.yolo import YOLO3DefaultValTransform
from gluoncv.data.transforms.presets.ssd import SSDDefaultTrainTransform
from gluoncv.data.transforms.presets.ssd import SSDDefaultValTransform
from gluoncv.utils.metrics.voc_detection import VOC07MApMetric
from gluoncv.utils.metrics.coco_detection import COCODetectionMetric
import gluoncv.data.transforms.bbox as tbbox
import gluoncv.data.transforms.image as timage
import gluoncv.data.transforms.experimental as experimental
import cv2
from gluoncv.utils.bbox import bbox_iou 
# from mxnet.contrib import amp

import neptune
os.environ['MXNET_CUDNN_AUTOTUNE_DEFAULT'] = '0'
os.environ['NEPTUNE_API_TOKEN'] = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiM2UxMWNkZTgtODZhZi00MWFmLWE3ZDEtOWYwZTliMzk4ZDFmIn0='


In [11]:
class SSDCustomValTransform(object):
    """Default SSD training transform which includes tons of image augmentations."""

    def __init__(self, width, height, anchors=None, mean=(0.485, 0.456, 0.406),
                 std=(0.229, 0.224, 0.225), iou_thresh=0.5, box_norm=(0.1, 0.1, 0.2, 0.2),
                 **kwargs):
        self._width = width
        self._height = height
        self._anchors = anchors
        self._mean = mean
        self._std = std
        if anchors is None:
            return

        # since we do not have predictions yet, so we ignore sampling here
        from gluoncv.model_zoo.ssd.target import SSDTargetGenerator
        self._target_generator = SSDTargetGenerator(
            iou_thresh=iou_thresh, stds=box_norm, negative_mining_ratio=-1, **kwargs)

    def __call__(self, src, label):
        """Apply transform to validation image/label."""
        # resize with random interpolation
        h, w, _ = src.shape
        img = timage.imresize(src, self._width, self._height, interp=9)
        bbox = tbbox.resize(label, (w, h), (self._width, self._height))

        # to tensor
        img = mx.nd.image.to_tensor(img)
        img = mx.nd.image.normalize(img, mean=self._mean, std=self._std)

        if self._anchors is None:
            return img, bbox.astype(img.dtype)

        # generate training target so cpu workers can help reduce the workload on gpu
        gt_bboxes = mx.nd.array(bbox[np.newaxis, :, :4])
        gt_ids = mx.nd.array(bbox[np.newaxis, :, 4:5])
        cls_targets, box_targets, _ = self._target_generator(
            self._anchors, None, gt_bboxes, gt_ids)
        return img, cls_targets[0], box_targets[0]

# SSD and YOLO

In [12]:
class training_network():
    def __init__(self, model='ssd300', ctx='gpu', resume_training=False, batch_size=4, num_workers=2, lr=0.001, 
                 lr_decay=0.1, lr_decay_epoch='60, 80', wd=0.0005, momentum=0.9, start_epoch=0,
                 epochs=2, dataset='voc', network='vgg16_atrous', resume='',
                 beta1=0.9, beta2=0.999, epsilon=1e-08, validation_threshold=0.5, nms_threshold=0.5, optimizer='sgd', x_id=None):
        
        # exp=False
        """
        Script responsible for training the class

        Arguments:
            val_interval (int, default: 1): Epoch interval for validation, increase the number will reduce the
                training time if validation is slow.
            wd (float, default: 0.0005): Weight decay, default is 5e-4
            momentum (float, default:0.9): SGD momentum, default is 0.9
            lr_decay_epoch (str, default: '60, 80'): epoches at which learning rate decays. default is 60, 80.
            lr_decay (float, default: 0.1): decay rate of learning rate. default is 0.1.
            lr (float, default: 0.001): Learning rate, default is 0.001
            start_epoch (int, default: 0): Starting epoch for resuming, default is 0 for new training. You can
                specify it to 100 for example to start from 100 epoch.
            resume (str, default: ''): Resume from previously saved parameters if not None. For example, you 
                can resume from ./ssd_xxx_0123.params'
            epochs (int, default:2): Training epochs.
            num_worker (int, default: 2): number to accelerate data loading
            dataset (str, default:'voc'): Training dataset. Now support voc.
            batch_size (int, default: 4): Training mini-batch size
            data_shape (int, default: 300): Input data shape, use 300, 512.
            network (str, default:'vgg16_atrous'): Base network name which serves as feature extraction base.
        """

        # amp.init()

        # TRAINING PARAMETERS
        self.net_type='SSD_or_YOLO'
        self.x_id = x_id
        self.old_save = None
        self.resume_training = resume_training
        self.batch_size=batch_size
        self.num_workers=num_workers
        self.learning_rate = lr
        self.weight_decay = wd
        self.momentum = momentum
        self.optimizer = optimizer
        self.lr_decay = lr_decay
        self.lr_decay_epoch = lr_decay_epoch
        self.start_epoch = start_epoch
        self.epochs = epochs
        self.resume = resume
        self.validation_threshold = validation_threshold
        self.nms_threshold = nms_threshold
        # self.experiment = experiment
        self.beta1=beta1
        self.beta2=beta2
        self.epsilon=epsilon
        self.best_map = 0
        self.model = model.lower()

        if ctx == 'cpu':
            self.ctx = [mx.cpu()]
        elif ctx == 'gpu':
            self.ctx = [mx.gpu(0)]
        else:
            raise ValueError('Invalid context.')
            
        # fix seed for mxnet, numpy and python builtin random generator.
        gutils.random.seed(233)

        if model.lower() == 'ssd_300_vgg16_atrous_voc':
            self.network = 'ssd'
            self.net_type = 'ssd'
            self.width, self.height = 300, 300
        elif model.lower() == 'ssd_512_resnet50_v1_voc':
            self.network = 'ssd'
            self.net_type = 'ssd'
            self.width, self.height = 512, 512
        elif model.lower() == 'ssd_512_vgg16_atrous_voc':
            self.network = 'ssd'
            self.net_type = 'ssd'
            self.width, self.height = 512, 512
        elif model.lower() == 'ssd_300_vgg16_atrous_coco':
            self.network = 'ssd'
            self.net_type = 'ssd'
            self.width, self.height = 300, 300
        elif model.lower() == 'ssd_512_vgg16_atrous_coco':
            self.network = 'ssd'
            self.net_type = 'ssd'
            self.width, self.height = 512, 512
        elif model.lower() == 'ssd_512_resnet50_v1_coco':
            self.network = 'ssd'
            self.net_type = 'ssd'
            self.width, self.height = 512, 512
        elif model.lower() == 'yolo3_darknet53_voc':
            self.network = 'yolo'
            self.net_type= 'yolo'
            self.width, self.height = 320, 320
        elif model.lower() == 'yolo3_darknet53_coco':
            self.network = 'yolo'
            self.net_type= 'yolo'
            self.width, self.height = 320, 320
        else:
            raise ValueError('Invalid model `{}`.'.format(model.lower()))
                
        # TODO: Specify the checkpoints save path
        self.save_prefix = 'drive/My Drive/UFBA/Doutorado/Ssd_test/checkpoints'
        
        # train and val rec file
        self.train_file = 'drive/My Drive/UFBA/Doutorado/Ssd_test/Dataset/train_teste_7_300_300.rec'
        self.val_file = 'drive/My Drive/UFBA/Doutorado/Ssd_test/Dataset/val_teste_7_300_300.rec'

        self.classes = ['bar_clamp', 'gear_box', 'vase', 'part_1', 'part_3', 'nozzle', 'pawn', 'turbine_housing'] # please, follow the order of the config.json file
        print('Classes: ', self.classes)

        # pretrained or pretrained_base?
        # pretrained (bool or str) – Boolean value controls whether to load the default 
        # pretrained weights for model. String value represents the hashtag for a certain 
        # version of pretrained weights.
        # pretrained_base (bool or str, optional, default is True) – Load pretrained base 
        # network, the extra layers are randomized. Note that if pretrained is True, this
        # has no effect.
        self.net = get_model(model, pretrained=True, norm_layer=gluon.nn.BatchNorm)
        self.net.reset_class(self.classes)
        
        # Initialize the weights
        if self.resume_training:
            self.net.initialize(force_reinit=True, ctx=self.ctx)
            self.net.load_params(self.resume, ctx=self.ctx)
        else:
            for param in self.net.collect_params().values():
                if param._data is not None:
                    continue
                param.initialize()

    def get_dataset(self):
        validation_threshold = self.validation_threshold
        self.train_dataset = gdata.RecordFileDetection(self.train_file)
        self.val_dataset = gdata.RecordFileDetection(self.val_file)
        # we are only using VOCMetric for evaluation
        if not self.net_type =='faster':
          self.val_metric = VOC07MApMetric(iou_thresh=validation_threshold, class_names=self.net.classes)
        else: 
          self.val_metric = VOC07MApMetric(iou_thresh=validation_threshold, class_names=self.classes)


    def show_summary(self):
        self.net.summary(mx.nd.ones((1, 3, self.height, self.width)))

    def get_dataloader(self):
        width, height = self.width, self.height
        train_dataset = self.train_dataset
        val_dataset = self.val_dataset
        batch_size = self.batch_size
        num_workers = self.num_workers
        network = self.network
        if network == 'ssd':
            # use fake data to generate fixed anchors for target generation
            with autograd.train_mode():
                _, _, anchors = self.net(mx.nd.zeros((1, 3, height, width)))

            batchify_fn = Tuple(Stack(), Stack(), Stack())  # stack image, cls_targets, box_targets
            train_loader = gluon.data.DataLoader(train_dataset.transform(SSDDefaultTrainTransform(width, height, anchors)),
                                                 batch_size, True, 
                                                 batchify_fn=batchify_fn, 
                                                 last_batch='rollover', 
                                                 num_workers=num_workers)

            # Val verdadeiro
            val_batchify_fn = Tuple(Stack(), Pad(pad_val=-1))
            val_loader = gluon.data.DataLoader(val_dataset.transform(SSDDefaultValTransform(width, height)),
                                               batch_size, False, 
                                               batchify_fn=val_batchify_fn, 
                                               last_batch='keep', 
                                               num_workers=num_workers)
          
            # use fake data to generate fixed anchors for target generation
            with mx.Context(mx.gpu(0)):
                anchors2 = anchors

            val_loader_loss = gluon.data.DataLoader(val_dataset.transform(SSDCustomValTransform(width, height, anchors2)),
                                                    batch_size, True, 
                                                    batchify_fn=batchify_fn, 
                                                    last_batch='rollover', 
                                                    num_workers=num_workers)
            self.val_loader_loss = val_loader_loss
        elif network == 'yolo':
            batchify_fn = Tuple(*([Stack() for _ in range(6)] + [Pad(axis=0, pad_val=-1) for _ in range(1)]))  # stack image, all targets generated
            # if args.no_random_shape:
            train_loader = gluon.data.DataLoader(train_dataset.transform(YOLO3DefaultTrainTransform(width, height, self.net)),
                                                 batch_size, True, 
                                                 batchify_fn=batchify_fn, 
                                                 last_batch='rollover', 
                                                 num_workers=num_workers)

            val_batchify_fn = Tuple(Stack(), Pad(pad_val=-1))
            val_loader = gluon.data.DataLoader(val_dataset.transform(YOLO3DefaultValTransform(width, height)),
                                               batch_size, False, 
                                               batchify_fn=val_batchify_fn, 
                                               last_batch='keep', 
                                               num_workers=num_workers)
        else:
            raise ValueError("Network {} not implemented".format(network))

        self.val_loader = val_loader
        self.train_loader = train_loader

    def save_params(self, current_map, epoch):
        prefix = self.save_prefix #self.save_prefix
        pre = '{:s}/{:s}/{:s}/'.format(prefix, self.net_type, self.x_id)
        print(prefix)
        best_map = self.best_map

        if not os.path.exists(pre):
          os.makedirs(pre)

        current_map = float(current_map)        
        if current_map > best_map:
            best_map = current_map
            cur_save= '{:s}{:s}_BE_{:04d}_map_{:.4f}.params'.format(pre, self.model, epoch, current_map) 
            print(cur_save)
            self.net.save_parameters(cur_save)
            if not (self.old_save == None):
              open(self.old_save, 'w').close() #overwrite and make the file blank instead - ref: https://stackoverflow.com/a/4914288/3553367
              os.remove(self.old_save) #delete the blank file from google drive will move the file to bin instead
            self.old_save = (cur_save)
        
        self.best_map = best_map
        print('Best map: ', self.best_map)

    def val_loss(self):
        """Training pipeline"""        
        val_data = self.val_loader_loss
        ctx = self.ctx
        val_metric = self.val_metric

        mbox_loss = gcv.loss.SSDMultiBoxLoss()
        ce_metric = mx.metric.Loss('CrossEntropy')
        smoothl1_metric = mx.metric.Loss('SmoothL1')

        ce_metric.reset() # Resets the internal evaluation result to initial state.
        smoothl1_metric.reset() # Resets the internal evaluation result to initial state.

        for i, batch in enumerate(val_data):
            batch_size = batch[0].shape[0]
            data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
            cls_targets = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0)
            box_targets = gluon.utils.split_and_load(batch[2], ctx_list=ctx, batch_axis=0)

            cls_preds = []
            box_preds = []
            for x in data:
                cls_pred, box_pred, _ = self.net(x)
                cls_preds.append(cls_pred)
                box_preds.append(box_pred)
                # descobrir o id de cada ifnerência pra usar no iou
            
            sum_loss, cls_loss, box_loss = mbox_loss(
                cls_preds, box_preds, cls_targets, box_targets)
        
        ce_metric.update(0, [l * batch_size for l in cls_loss])
        smoothl1_metric.update(0, [l * batch_size for l in box_loss])

        name1, loss1 = ce_metric.get()
        name2, loss2 = smoothl1_metric.get()

        return name1, loss1, name2, loss2

    def validate(self):
        """Test on validation dataset."""
        val_data = self.val_loader
        ctx = self.ctx
        val_metric = self.val_metric
        nms_threshold = self.nms_threshold
        validation_threshold = self.validation_threshold

        val_metric.reset()
        # set nms threshold and topk constraint
        # post_nms = maximum number of objects per image
        self.net.set_nms(nms_thresh=nms_threshold, nms_topk=200, post_nms=len(self.classes)) # default: iou=0.45 e topk=400

        # allow the MXNet engine to perform graph optimization for best performance.
        self.net.hybridize(static_alloc=True, static_shape=True)

        num_of_classes = len(self.classes)
        # total number of correct prediction by class
        tp = [0] * num_of_classes
        # false positives by class
        fp = [0] * num_of_classes
        # count the number of gt by class
        gt_by_class = [0] * num_of_classes
        # rec and prec by class
        rec_by_class = [0] * num_of_classes
        prec_by_class = [0] * num_of_classes
        confusion_matrix = np.zeros((num_of_classes, num_of_classes))

        for batch in val_data:
            batch_size = batch[0].shape[0]
            data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
            label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)

            pred_bboxes_list = []
            pred_label_list = []
            pred_scores_list = []
            gt_bboxes_list = []
            gt_label_list = []
            
            for x, y in zip(data, label):
                # get prediction results
                ids, scores, bboxes = self.net(x)
                pred_label_list.append(ids)
                pred_scores_list.append(scores)
                # clip to image size
                pred_bboxes_list.append(bboxes.clip(0, batch[0].shape[2]))
                # split ground truths
                gt_label_list.append(y.slice_axis(axis=-1, begin=4, end=5))
                gt_bboxes_list.append(y.slice_axis(axis=-1, begin=0, end=4))

            # Uncomment the following line if you want to plot the images in each inference to visually  check the tp, fp and fn 
            # self.show_images(x, pred_label_list, pred_bboxes_list, gt_label_list, gt_bboxes_list)
            
            # update metric
            val_metric.update(pred_bboxes_list, pred_label_list, pred_scores_list, gt_bboxes_list, gt_label_list) #, gt_difficults)
            
            # Get Micro Averaging (precision and recall by each class) in each batch
            for img in range(batch_size):
                # count +1 for this class id. It will get the total number of gt by class
                # It is useful when considering unbalanced datasets
                for gt_idx in gt_label_list[0][img]:
                    index = int(gt_idx.asnumpy()[0])
                    gt_by_class[index] += 1
            
                for (pred_label, pred_bbox) in zip(pred_label_list[0][img], list(pred_bboxes_list[0][img])):
                    pred_label = int(pred_label.asnumpy()[0])
                    pred_bbox = pred_bbox.asnumpy()
                    pred_bbox = np.expand_dims(pred_bbox, axis=0)
                    match = 0
                    for (gt_bbox_label, gt_bbox_coordinates) in zip(gt_label_list[0][img], list(gt_bboxes_list[0][img])):
                        gt_bbox_coord = gt_bbox_coordinates.asnumpy()
                        gt_bbox_coord = np.expand_dims(gt_bbox_coord, axis=0)
                        gt_bbox_label = int(gt_bbox_label.asnumpy()[0])
                        iou = bbox_iou(pred_bbox, gt_bbox_coord)
                        
                        # Correct inference
                        if iou > validation_threshold and pred_label == gt_bbox_label:
                            confusion_matrix[gt_bbox_label][pred_label] += 1
                            tp[gt_bbox_label] += 1 # Correct classification
                            match = 1
                        # Incorrect inference - missed the correct class but put the bounding box in other class
                        elif iou > validation_threshold:
                            confusion_matrix[gt_bbox_label][pred_label] += 1
                            fp[pred_label] += 1
                            match = 1
                        
                    if not match:
                        fp[pred_label] += 1
                                
        # calculate the Recall and Precision by class
        tp = np.array(tp) # we can also sum the matrix diagonal
        fp = np.array(fp)
        
        fp_sum = sum(fp)
        tp_sum = sum(tp)

        # rec and prec according to the micro averaging
        for i, (gt_value, tp_value) in enumerate(zip(gt_by_class, tp)):
            rec_by_class[i] += tp_value/gt_value
            # If an element of fp + tp is 0,
            # the corresponding element of prec[l] is nan.
            with np.errstate(divide='ignore', invalid='ignore'):
                prec_by_class[i] += tp_value/(tp_value+fp[i])

        return val_metric.get(), rec_by_class, prec_by_class

    def create_optimizer(self):
        optimizer = self.optimizer
        momentum = self.momentum
        wd = self.weight_decay
        lr = self.learning_rate
        beta1 = self.beta1
        beta2 = self.beta2
        epsilon = self.epsilon
        
        if not self.net_type == 'faster':
          
          if optimizer.lower() == 'sgd':
              # wd: The weight decay (or L2 regularization) coefficient.
              self.trainer = gluon.Trainer(self.net.collect_params(), optimizer,
                                      {'learning_rate': lr, 'wd': wd, 'momentum': momentum})
          elif optimizer.lower() == 'adam':
              self.trainer = gluon.Trainer(self.net.collect_params(), optimizer,
                                      {'learning_rate': lr, 'beta1': beta1, 'beta2': beta2, 
                                      'epsilon': epsilon})
        else:
          if optimizer == 'sgd':
            optimizer_params = {'multi_precision' : True, 'learning_rate': lr, 
                                          'wd': wd, 'momentum': momentum}
            # wd: The weight decay (or L2 regularization) coefficient.
            
          elif optimizer == 'adam':
              optimizer_params = {'multi_precision' : True, 'learning_rate': lr, 'beta1': beta1, 'beta2': beta2,
                                            'epsilon': epsilon}
      
          self.trainer = gluon.Trainer(self.net.collect_train_params(), optimizer, optimizer_params
                                      , update_on_kvstore=(False if False else None), kvstore=self.kv) #self.amp
          

    def validate_main(self, epoch):
        # consider reduce the frequency of validation to save time
        (map_name, mean_ap), rec_by_class, prec_by_class = self.validate()
        val_msg = '\n'.join(['{}={}'.format(k, v) for k, v in zip(map_name, mean_ap)])
        
        for i, class_name in enumerate(self.classes):
            experiment.log_metric('rec_by_class_val_' + class_name, epoch, rec_by_class[i])
            experiment.log_metric('prec_by_class_val_' + class_name, epoch, prec_by_class[i])
        
        for k, v in zip(map_name, mean_ap):
            experiment.log_metric('map_' + k, epoch, v)
        
        print('[Epoch {}] Validation: \n{}'.format(epoch, val_msg))
        current_map = float(mean_ap[-1])
        
        self.save_params(current_map, epoch)    

    def ssd_train(self):
        """Training pipeline"""
        ctx = self.ctx
        train_data = self.train_loader
        start_epoch = self.start_epoch
        epochs = self.epochs
        # experiment = self.experiment
        trainer = self.trainer
        optimizer = self.optimizer

        mbox_loss = gcv.loss.SSDMultiBoxLoss()
        ce_metric = mx.metric.Loss('CrossEntropy')
        smoothl1_metric = mx.metric.Loss('SmoothL1')

        if optimizer =='sgd':
            # lr decay policy
            lr_decay = float(lr_decay)
            lr_steps = sorted([float(ls) for ls in lr_decay_epoch.split(',') if ls.strip()]) 

        print('Start training from [Epoch {}]'.format(start_epoch))
        start_train_time = time.time()
        for epoch in range(start_epoch, epochs):
            start_epoch_time = time.time()
            experiment.log_metric('learning_rate', epoch, trainer.learning_rate)

            if optimizer == 'sgd':
              while lr_steps and epoch >= lr_steps[0]:
                  new_lr = trainer.learning_rate * lr_decay
                  lr_steps.pop(0) # removes the first element in the list
                  trainer.set_learning_rate(new_lr) # Set a new learning rate
                  print("[Epoch {}] Set learning rate to {}".format(epoch, new_lr))
                
            ce_metric.reset() # Resets the internal evaluation result to initial state.
            smoothl1_metric.reset() # Resets the internal evaluation result to initial state.

            tic = time.time() # each epoch time in seconds
            btic = time.time() # each batch time interval in seconds
                
            # Activates or deactivates HybridBlocks recursively. it speeds up the training process
            self.net.hybridize(static_alloc=True, static_shape=True)
                
            for i, batch in enumerate(train_data):
                # Wait for completion of previous iteration to
                # avoid unnecessary memory allocation
                # nd.waitall()

                batch_size = batch[0].shape[0]
                data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
                cls_targets = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0)
                box_targets = gluon.utils.split_and_load(batch[2], ctx_list=ctx, batch_axis=0)

                with autograd.record():
                    cls_preds = []
                    box_preds = []
                    for x in data:
                        cls_pred, box_pred, _ = self.net(x)
                        cls_preds.append(cls_pred)
                        box_preds.append(box_pred)
                        
                    sum_loss, cls_loss, box_loss = mbox_loss(
                        cls_preds, box_preds, cls_targets, box_targets)
                        
                    # with amp.scale_loss(sum_loss, trainer) as scaled_loss:
                        # autograd.backward(scaled_loss)
                    autograd.backward(sum_loss)
                    
                # since we have already normalized the loss, we don't want to normalize
                # by batch-size anymore
                trainer.step(1)
                ce_metric.update(0, [l * batch_size for l in cls_loss])
                smoothl1_metric.update(0, [l * batch_size for l in box_loss])

                name1, loss1 = ce_metric.get()
                name2, loss2 = smoothl1_metric.get()
                print('[Epoch {}][Batch {}], Speed: {:.3f} samples/sec, {}={:.3f}, {}={:.3f}'.format(
                    epoch, i, batch_size/(time.time()-btic), name1, loss1, name2, loss2))
                btic = time.time()

            # log the epoch info
            name1, loss1 = ce_metric.get()
            name2, loss2 = smoothl1_metric.get()
            experiment.log_metric('cross_entropy_training_loss', epoch, loss1)
            experiment.log_metric('smooth_l1_training_loss', epoch, loss2) 
            experiment.log_metric('train_sum_loss', epoch, loss1 + loss2) 

            print('[Epoch {}] - Time (min): {:.3f}, {}={:.3f}, {}={:.3f}'.format(
                epoch, (time.time()-tic)/60, name1, loss1, name2, loss2))

            # log SSD LOSS            
            val_name1, val_loss1, val_name2, val_loss2 = self.val_loss()
            current_val_loss = val_loss1 + val_loss2
            experiment.log_metric('cross_entropy_validation_loss', epoch, val_loss1)
            experiment.log_metric('smooth_l1_validation_loss', epoch, val_loss2) 
            experiment.log_metric('validation_sum_loss', epoch, current_val_loss) 

            self.validate_main(epoch)
        
        # Displays the total time of the training
        print('Train time {:.3f}'.format(time.time() - start_train_time))
    
    def yolo_train(self):
        """Training pipeline"""
        ctx = self.ctx
        train_data = self.train_loader
        start_epoch = self.start_epoch
        epochs = self.epochs
        # experiment = self.experiment
        trainer = self.trainer
        optimizer = self.optimizer

        # metrics
        obj_metrics = mx.metric.Loss('ObjLoss')
        center_metrics = mx.metric.Loss('BoxCenterLoss')
        scale_metrics = mx.metric.Loss('BoxScaleLoss')
        cls_metrics = mx.metric.Loss('ClassLoss')

        if optimizer =='sgd':
            # lr decay policy
            lr_decay = float(lr_decay)
            lr_steps = sorted([float(ls) for ls in lr_decay_epoch.split(',') if ls.strip()]) 

        print('Start training from [Epoch {}]'.format(start_epoch))
        start_train_time = time.time()
        for epoch in range(start_epoch, epochs):
            experiment.log_metric('learning_rate', epoch, trainer.learning_rate)

            start_epoch_time = time.time()
            tic = time.time() # each epoch time in seconds
            btic = time.time() # each batch time interval in seconds
            mx.nd.waitall()
            self.net.hybridize()
            for i, batch in enumerate(train_data):
                data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
                # objectness, center_targets, scale_targets, weights, class_targets
                fixed_targets = [gluon.utils.split_and_load(batch[it], ctx_list=ctx, batch_axis=0) for it in range(1, 6)]
                gt_boxes = gluon.utils.split_and_load(batch[6], ctx_list=ctx, batch_axis=0)
                sum_loss = []
                obj_losses = []
                center_losses = []
                scale_losses = []
                cls_losses = []
                with autograd.record():
                    for ix, x in enumerate(data):
                        obj_loss, center_loss, scale_loss, cls_loss = self.net(x, gt_boxes[ix], *[ft[ix] for ft in fixed_targets])
                        sum_loss.append(obj_loss + center_loss + scale_loss + cls_loss)
                        obj_losses.append(obj_loss)
                        center_losses.append(center_loss)
                        scale_losses.append(scale_loss)
                        cls_losses.append(cls_loss)
                    # if args.amp:
                    # with amp.scale_loss(sum_loss, trainer) as scaled_loss:
                    #         autograd.backward(scaled_loss)
                    # else:
                    autograd.backward(sum_loss)
                trainer.step(self.batch_size)
                obj_metrics.update(0, obj_losses)
                center_metrics.update(0, center_losses)
                scale_metrics.update(0, scale_losses)
                cls_metrics.update(0, cls_losses)
                # if self.log_interval and not (i + 1) % self.log_interval:
                name1, loss1 = obj_metrics.get()
                name2, loss2 = center_metrics.get()
                name3, loss3 = scale_metrics.get()
                name4, loss4 = cls_metrics.get()
                print('[Epoch {}][Batch {}], LR: {:.2E}, Speed: {:.3f} samples/sec, {}={:.3f}, {}={:.3f}, {}={:.3f}, {}={:.3f}'.format(
                    epoch, i, trainer.learning_rate, self.batch_size/(time.time()-btic), name1, loss1, name2, loss2, name3, loss3, name4, loss4))
                btic = time.time()


            name1, loss1 = obj_metrics.get()
            name2, loss2 = center_metrics.get()
            name3, loss3 = scale_metrics.get()
            name4, loss4 = cls_metrics.get()
            experiment.log_metric('Obj_metrics', epoch, loss1)
            experiment.log_metric('Center_metrics', epoch, loss2)
            experiment.log_metric('Scale_metrics', epoch, loss3)
            experiment.log_metric('cls_metrics', epoch, loss4)
            print('[Epoch {}] Training cost: {:.3f}, {}={:.3f}, {}={:.3f}, {}={:.3f}, {}={:.3f}'.format(
                epoch, (time.time()-tic), name1, loss1, name2, loss2, name3, loss3, name4, loss4))

            self.validate_main(epoch)
        
        # Displays the total time of the training
        print('Train time {:.3f}'.format(time.time() - start_train_time))

    def faster_train(self):
        print("Train start")
        """Training pipeline"""
        self.kv_store = 'device' if (False and 'nccl' in self.kv_store) else self.kv_store #if (self.amp and 'nccl' in self.kv_store) else self.kv_store
        self.kv = mx.kvstore.create(self.kv_store)
        self.net.collect_params().setattr('grad_req', 'null')
        self.net.collect_train_params().setattr('grad_req', 'write')
        optimizer_params = {'learning_rate': self.lr, 'wd': self.wd, 'momentum': self.momentum}
        self.create_optimizer()
        trainer = self.trainer
        lr_decay = float(self.lr_decay)
        lr_steps = sorted([float(ls) for ls in self.lr_decay_epoch.split(',') if ls.strip()])
        lr_warmup = float(self.lr_warmup)  # avoid int division

        # TODO(zhreshold) losses?
        rpn_cls_loss = mx.gluon.loss.SigmoidBinaryCrossEntropyLoss(from_sigmoid=False)
        rpn_box_loss = mx.gluon.loss.HuberLoss(rho=self.rpn_smoothl1_rho)  # == smoothl1
        rcnn_cls_loss = mx.gluon.loss.SoftmaxCrossEntropyLoss()
        rcnn_box_loss = mx.gluon.loss.HuberLoss(rho=self.rcnn_smoothl1_rho)  # == smoothl1
        metrics = [mx.metric.Loss('RPN_Conf'),
                   mx.metric.Loss('RPN_SmoothL1'),
                   mx.metric.Loss('RCNN_CrossEntropy'),
                   mx.metric.Loss('RCNN_SmoothL1'), ]

        rpn_acc_metric = RPNAccMetric()
        rpn_bbox_metric = RPNL1LossMetric()
        rcnn_acc_metric = RCNNAccMetric()
        rcnn_bbox_metric = RCNNL1LossMetric()
        metrics2 = [rpn_acc_metric, rpn_bbox_metric, rcnn_acc_metric, rcnn_bbox_metric]

        print('Start training from [Epoch {}]'.format(self.start_epoch))#logger.info
        best_map = [0]
        for epoch in range(self.start_epoch, self.epochs):

            #self.experiment.log_metric('learning_rate', epoch, trainer.learning_rate)

            rcnn_task = ForwardBackwardTask(self.net, trainer, rpn_cls_loss, rpn_box_loss, rcnn_cls_loss,
                                            rcnn_box_loss, mix_ratio=1.0, amp_enabled=self.amp)
            executor = Parallel(self.executor_threads, rcnn_task) if not self.horovod else None
            mix_ratio = 1.0
            if not self.disable_hybridization:
                self.hybridize(static_alloc=self.static_alloc)
            if self.mixup:
                # TODO(zhreshold) only support evenly mixup now, target generator needs to be modified otherwise
                self.train_data._dataset._data.set_mixup(np.random.uniform, 0.5, 0.5)
                mix_ratio = 0.5
                if epoch >= self.epochs - self.no_mixup_epochs:
                    self.train_data._dataset._data.set_mixup(None)
                    mix_ratio = 1.0

            experiment.log_metric('learning_rate', epoch, trainer.learning_rate)
            if self.optimizer == 'sgd':
              while lr_steps and epoch >= lr_steps[0]:
                  new_lr = trainer.learning_rate * lr_decay
                  lr_steps.pop(0)
                  trainer.set_learning_rate(new_lr)
                  print("[Epoch {}] Set learning rate to {}".format(epoch, new_lr))#logger.info
            for metric in metrics:
                metric.reset()
            tic = time.time()
            btic = time.time()
            base_lr = trainer.learning_rate
            rcnn_task.mix_ratio = mix_ratio
            for i, batch in enumerate(self.train_data):
                if epoch == 0 and i <= lr_warmup:
                    # adjust based on real percentage
                    new_lr = base_lr * self.get_lr_at_iter(i / lr_warmup, self.lr_warmup_factor)
                    if new_lr != trainer.learning_rate:
                        if i % self.log_interval == 0:
                            print(
                                '[Epoch 0 Iteration {}] Set learning rate to {}'.format(i, new_lr))#logger.info
                        trainer.set_learning_rate(new_lr)
                batch = self.split_and_load(batch, ctx_list=self.ctx)
                metric_losses = [[] for _ in metrics]
                add_losses = [[] for _ in metrics2]
                if executor is not None:
                    for data in zip(*batch):
                        executor.put(data)
                for j in range(len(self.ctx)):
                    if executor is not None:
                        result = executor.get()
                    else:
                        result = rcnn_task.forward_backward(list(zip(*batch))[0])

                    for k in range(len(metric_losses)):
                        metric_losses[k].append(result[k])
                    for k in range(len(add_losses)):
                        add_losses[k].append(result[len(metric_losses) + k])
                for metric, record in zip(metrics, metric_losses):
                    metric.update(0, record)
                for metric, records in zip(metrics2, add_losses):
                    for pred in records:
                        metric.update(pred[0], pred[1])
                trainer.step(self.batch_size)

                # update metrics        #  (not self.horovod or hvd.rank() == 0) and  _k_
                if self.log_interval \
                        and not (i + 1) % self.log_interval:
                    msg = ','.join(
                        ['{}={:.3f}'.format(*metric.get()) for metric in metrics + metrics2])
                    print('[Epoch {}][Batch {}], Speed: {:.3f} samples/sec, {}'.format(
                        epoch, i, self.log_interval * self.batch_size / (time.time() - btic), msg)) #logger.info
                    btic = time.time()

            # if (not self.horovod) or hvd.rank() == 0:
            msg = ','.join(['{}={:.3f}'.format(*metric.get()) for metric in metrics])
            print('[Epoch {}] Training cost: {:.3f}, {}'.format(
                epoch, (time.time() - tic), msg)) #logger.info
            
            self.validate_main(epoch)#map_name, mean_ap = self.validate(self.net, self.val_data, self.ctx, self.val_metric)
 
        print("Traning... OK")    
            
    def train(self):
        lr_decay = self.lr_decay
        lr_decay_epoch = self.lr_decay_epoch        
        optimizer = self.optimizer
        network = self.network

        # Gluon-CV requires you to create and load the parameters of your model first on 
        # the CPU - so specify ctx=None - and when all that is done you move the 
        # whole model on the GPU with:
        self.net.collect_params().reset_ctx(self.ctx)

        # # First create the trainer. Obs: you should reset_ctx before creating the optimizer

        if self.net_type == 'ssd':
          # First create the trainer. Obs: you should reset_ctx before creating the optimizer
            self.create_optimizer()
            self.ssd_train()
        elif self.net_type == 'yolo':
          # First create the trainer. Obs: you should reset_ctx before creating the optimizer
            self.create_optimizer()
            self.yolo_train()
        elif self.net_type =='faster':
            self.faster_train()

# Faster RCNN

In [13]:
# """Train Faster-RCNN end to end."""

class Faster_rcnn(training_network):
    def __init__(self, model='faster_rcnn_resnet50_v1b_voc' ,network = 'resnet50_v1b', dataset = 'voc',num_workers = 4,batch_size = 1,gpu = 0,epochs = 20,
                 resume = '',start_epoch = 0,lr = 0.001,lr_decay = 0.1,lr_decay_epoch= '14,20',lr_warmup = -1, beta1=0.9, beta2=0.999, epsilon=1e-08,
                 lr_warmup_factor = 1. / 3.,momentum = 0.9, validation_threshold=0.5, nms_threshold=0.5,wd = 5e-4,log_interval = 1,save_interval = 1, optimizer = 'sgd',
                 val_interval = 1,exp = False,train_file = 'train.rec',val_file = 'val.rec',ctx = 'gpu',save_prefix = 'faster_cnn_model_', x_id=None):
        self.kv=None
        self.x_id = x_id
        self.old_save = None
        self.net_type = 'faster'
        self.batch_size=batch_size #
        self.num_workers=num_workers#
        self.learning_rate = lr#
        self.lr = lr#
        self.weight_decay = wd#
        self.wd = wd#
        self.momentum = momentum#
        self.optimizer = optimizer
        self.lr_decay = lr_decay#
        self.lr_decay_epoch = lr_decay_epoch#
        self.val_interval = val_interval#
        self.start_epoch = start_epoch#
        self.epochs = epochs#
        self.log_interval = log_interval#
        self.save_interval = save_interval#
        self.resume = resume#
        self.validation_threshold = validation_threshold
        self.nms_threshold = nms_threshold
        self.network = network#
        self.dataset = dataset#
        self.beta1=beta1
        self.beta2=beta2
        self.epsilon=epsilon
        self.trainer = None
        self.model = model.lower()

        self.gpu = gpu#
        
        self.lr_warmup = lr_warmup#
        self.lr_warmup_factor = lr_warmup_factor#
        
        self.mixup = False
        self.no_mixup_epochs = 20
        self.norm_layer = None
        self.rpn_smoothl1_rho = 1. / 9.
        self.rcnn_smoothl1_rho = 1
        self.use_fpn = False
        self.disable_hybridization = True
        self.static_alloc = True
        self.amp = False
        self.horovod = False
        self.executor_threads = 2
        self.kv_store = 'nccl'
        self.experiment = exp

        self.save_prefix = save_prefix
        self.train_file = train_file
        self.val_file = val_file
        self.validation_threshold = 0.5
        self.train_dataset = None
        self.val_dataset = None
        self.val_metric = None
        self.classes = ['bar_clamp', 'gear_box', 'vase', 'part_1', 'part_3', 'nozzle', 'pawn', 'turbine_housing']
        #Configuração da rede

        self.nms_thresh = 0.5
        self.nms_topk = -1
        self.post_nms = -1

        self.max_num_gt = 100
        self.best_map=0

        if ctx == 'cpu':
            self.ctx = [mx.cpu()]
        elif ctx == 'gpu':
            self.ctx = [mx.gpu(self.gpu)]
        else:
            raise ValueError('Invalid context.')

        # fix seed for mxnet, numpy and python builtin random generator.
        gutils.random.seed(233)

        self.net = None

        self.get_dataset()
        self.create_network()
        self.data_loader()

    def create_network(self):
        kwargs = {}
        module_list = []
        if self.use_fpn:
            module_list.append('fpn')
        if self.norm_layer is not None:
            module_list.append(self.norm_layer)
            if self.norm_layer == 'syncbn':
                kwargs['num_devices'] = len(self.ctx)

        num_gpus =len(self.ctx) # hvd.size() if self.horovod else __k__
        net_name = '_'.join(('faster_rcnn', *module_list, self.network, self.dataset))
        
        self.net = get_model(net_name, pretrained_base=True,
                        per_device_batch_size=self.batch_size // num_gpus, **kwargs)
        # self.save_prefix += net_name
        if self.resume.strip():
            self.net.load_parameters(self.resume.strip())
        else:
            for param in self.net.collect_params().values():
                if param._data is not None:
                    continue
                param.initialize()
        self.net.collect_params().reset_ctx(self.ctx)
        print("Network... OK")

    def data_loader(self):
        """Get dataloader."""
        train_transform = FasterRCNNDefaultTrainTransform
        val_transform = FasterRCNNDefaultValTransform
        train_bfn = FasterRCNNTrainBatchify(self.net, len(self.ctx))
        if hasattr(self.train_dataset, 'get_im_aspect_ratio'):
            im_aspect_ratio = self.train_dataset.get_im_aspect_ratio()
        else:
            im_aspect_ratio = [1.] * len(self.train_dataset)
        train_sampler = \
            gcv.nn.sampler.SplitSortedBucketSampler(im_aspect_ratio, self.batch_size,
                                                    num_parts=1, #hvd.size() if self.horovod else _k_
                                                    part_index=0, #hvd.rank() if self.horovod else _k_ 
                                                    shuffle=True)
        self.train_data = mx.gluon.data.DataLoader(self.train_dataset.transform(
            train_transform(self.net.short, self.net.max_size, self.net, ashape=self.net.ashape, multi_stage=self.use_fpn)),
            batch_sampler=train_sampler, batchify_fn=train_bfn, num_workers=self.num_workers)
        val_bfn = Tuple(*[Append() for _ in range(3)])
        short = self.net.short[-1] if isinstance(self.net.short, (tuple, list)) else self.net.short
        # validation use 1 sample per device
        self.val_data = mx.gluon.data.DataLoader(
            self.val_dataset.transform(val_transform(short, self.net.max_size)), len(self.ctx), False,
            batchify_fn=val_bfn, last_batch='keep', num_workers=self.num_workers)
        print("Dataloader... OK")

    def get_lr_at_iter(self,alpha, lr_warmup_factor=1. / 3.):
        return lr_warmup_factor * (1 - alpha) + alpha

    def split_and_load(self,batch, ctx_list):
        """Split data to 1 batch each device."""
        new_batch = []
        for i, data in enumerate(batch):
            if isinstance(data, (list, tuple)):
                new_data = [x.as_in_context(ctx) for x, ctx in zip(data, ctx_list)]
            else:
                new_data = [data.as_in_context(ctx_list[0])]
            new_batch.append(new_data)
        return new_batch
    

#FASTER  ##,net, val_data, ctx, val_metric
    def validate(self):
        """Test on validation dataset."""
        val_data = self.val_data
        ctx = self.ctx
        val_metric = self.val_metric
        nms_threshold = self.nms_threshold
        validation_threshold = self.validation_threshold

        clipper = gcv.nn.bbox.BBoxClipToImage()
        val_metric.reset()
        # if not self.disable_hybridization:
            # input format is differnet than training, thus rehybridization is needed.
        # self.net.hybridize(static_alloc=self.static_alloc)
        for batch in val_data:
            batch = self.split_and_load(batch, ctx_list=ctx)
            pred_bboxes_list = []
            pred_label_list = []
            pred_scores_list = []
            gt_bboxes_list = []
            gt_label_list = []
            # gt_difficults = []
            for x, y, im_scale in zip(*batch):
                # get prediction results
                ids, scores, bboxes = self.net(x)
                pred_label_list.append(ids)
                pred_scores_list.append(scores)
                # clip to image size
                pred_bboxes_list.append(clipper(bboxes, x))
                # rescale to original resolution
                im_scale = im_scale.reshape((-1)).asscalar()
                pred_bboxes_list[-1] *= im_scale
                # split ground truths
                gt_label_list.append(y.slice_axis(axis=-1, begin=4, end=5))
                gt_bboxes_list.append(y.slice_axis(axis=-1, begin=0, end=4))
                gt_bboxes_list[-1] *= im_scale

            # update metric
            for pred_bbox, pred_id, pred_score, gt_bbox, gt_id in zip(pred_bboxes_list, pred_label_list,
                                                                            pred_scores_list, gt_bboxes_list,
                                                                            gt_label_list):
                val_metric.update(pred_bbox, pred_id, pred_score, gt_bbox, gt_id)

        num_of_classes = len(self.classes)
        # total number of correct prediction by class
        tp = [0] * num_of_classes
        # false positives by class
        fp = [0] * num_of_classes
        # count the number of gt by class
        gt_by_class = [0] * num_of_classes
        # rec and prec by class
        rec_by_class = [0] * num_of_classes
        prec_by_class = [0] * num_of_classes
        confusion_matrix = np.zeros((num_of_classes, num_of_classes))
        for img in range(self.batch_size):
              # count +1 for this class id. It will get the total number of gt by class
              # It is useful when considering unbalanced datasets
              for gt_idx in gt_label_list[0][img]:
                  index = int(gt_idx.asnumpy()[0])
                  gt_by_class[index] += 1
          
              for (pred_label, pred_bbox) in zip(pred_label_list[0][img], list(pred_bboxes_list[0][img])):
                  pred_label = int(pred_label.asnumpy()[0])
                  pred_bbox = pred_bbox.asnumpy()
                  pred_bbox = np.expand_dims(pred_bbox, axis=0)
                  match = 0
                  for (gt_bbox_label, gt_bbox_coordinates) in zip(gt_label_list[0][img], list(gt_bboxes_list[0][img])):
                      gt_bbox_coord = gt_bbox_coordinates.asnumpy()
                      gt_bbox_coord = np.expand_dims(gt_bbox_coord, axis=0)
                      gt_bbox_label = int(gt_bbox_label.asnumpy()[0])
                      iou = bbox_iou(pred_bbox, gt_bbox_coord)
                      
                      # Correct inference
                      if iou > validation_threshold and pred_label == gt_bbox_label:
                          confusion_matrix[gt_bbox_label][pred_label] += 1
                          tp[gt_bbox_label] += 1 # Correct classification
                          match = 1
                      # Incorrect inference - missed the correct class but put the bounding box in other class
                      elif iou > validation_threshold:
                          confusion_matrix[gt_bbox_label][pred_label] += 1
                          fp[pred_label] += 1
                          match = 1
                      
                  if not match:
                      fp[pred_label] += 1
                              
        # calculate the Recall and Precision by class
        tp = np.array(tp) # we can also sum the matrix diagonal
        fp = np.array(fp)
        fp_sum = sum(fp)
        tp_sum = sum(tp)

        # rec and prec according to the micro averaging
        for i, (gt_value, tp_value) in enumerate(zip(gt_by_class, tp)):
            rec_by_class[i] += tp_value/gt_value
            # If an element of fp + tp is 0,
            # the corresponding element of prec[l] is nan.
            with np.errstate(divide='ignore', invalid='ignore'):
                prec_by_class[i] += tp_value/(tp_value+fp[i])

        return val_metric.get(), rec_by_class, prec_by_class


# Função Main 

In [14]:
if __name__ == '__main__':
  try:
    neptune.init('caioviturino/IJR2020')
    #'yolo3_darknet53_coco'
    #'faster_rcnn_resnet50_v1b_voc'
    PARAMS = {'model_name': 'faster_rcnn_resnet50_v1b_voc', 
              'ctx': 'gpu',
              'lr_decay_epoch': '30,50',
              'lr': 1e-3, # 0.0001,
              'lr_decay': 0.1,
              'batch_size': 16, #Does not affect faster_rcnn, once in Google Colab it only supports batch size = 1 due to available memory
              'epochs': 60, #80
              'optimizer': 'adam', #https://mxnet.apache.org/versions/1.6/api/python/docs/tutorials/packages/optimizer/index.html
              'wd': 5e-4, # 0.0005, # sgd parameter
              'momentum': 0.9, # 0.9, # sgd parameter
              'beta1': 0.9, # 0.9, # adam parameter
              'beta2': 0.999, #0.999, # adam parameter
              'epsilon': 1e-08, # 1e-08, # adam parameter
              'validation_threshold': 0.5,
              'nms_threshold': 0.5
              }

    # create experiment (all parameters are optional)
    experiment = neptune.create_experiment(name=PARAMS['model_name'],
                                           params=PARAMS,
                                           tags=[PARAMS['model_name'], PARAMS['optimizer'], 'kleber'])
    
    
    print('Experiment ID = ' + experiment.id + '\n Net= '+ PARAMS['model_name'])


    ## TODO : Trocar == por *contains*
    
    if PARAMS['model_name'] == 'faster_rcnn_resnet50_v1b_voc':
      train_loc = 'drive/My Drive/UFBA/Doutorado/Ssd_test/Dataset/train_teste_7_300_300.rec'
      val_loc = 'drive/My Drive/UFBA/Doutorado/Ssd_test/Dataset/val_teste_7_300_300.rec'
      save_loc = 'drive/My Drive/UFBA/Doutorado/Ssd_test/checkpoints'
      fast = Faster_rcnn(model=PARAMS['model_name'], train_file=train_loc, val_file=val_loc,save_prefix=save_loc, ctx=PARAMS['ctx'], \
                         lr_decay_epoch=PARAMS['lr_decay_epoch'], lr=PARAMS['lr'], \
                         lr_decay=PARAMS['lr_decay'], batch_size=1, epochs=PARAMS['epochs'], \
                         optimizer=PARAMS['optimizer'],  \
                         validation_threshold=PARAMS['validation_threshold'], nms_threshold=PARAMS['nms_threshold'], \
                         beta1=PARAMS['beta1'], beta2=PARAMS['beta2'], epsilon=PARAMS['epsilon'], \
                         wd=PARAMS['wd'], momentum=PARAMS['momentum'], x_id = experiment.id) 
                         #exp = experiment
      # print('\n'.join(str(a) for a in dir(fast)))
      fast.faster_train()
      os.environ['MXNET_CUDNN_AUTOTUNE_DEFAULT'] = '0'
      os.environ['MXNET_GPU_MEM_POOL_TYPE'] = 'Round'
      os.environ['MXNET_GPU_MEM_POOL_ROUND_LINEAR_CUTOFF'] = '26'
      os.environ['MXNET_EXEC_BULK_EXEC_MAX_NODE_TRAIN_FWD'] = '999'
      os.environ['MXNET_EXEC_BULK_EXEC_MAX_NODE_TRAIN_BWD'] = '25'
      os.environ['MXNET_GPU_COPY_NTHREADS'] = '1'
      os.environ['MXNET_OPTIMIZER_AGGREGATION_SIZE'] = '54'
    else:
      train_object = training_network(model=PARAMS['model_name'], ctx=PARAMS['ctx'], \
                                          lr_decay_epoch=PARAMS['lr_decay_epoch'], lr=PARAMS['lr'], \
                                          lr_decay=PARAMS['lr_decay'], batch_size=PARAMS['batch_size'], epochs=PARAMS['epochs'], \
                                          optimizer=PARAMS['optimizer'],  \
                                          validation_threshold=PARAMS['validation_threshold'], nms_threshold=PARAMS['nms_threshold'], \
                                          beta1=PARAMS['beta1'], beta2=PARAMS['beta2'], epsilon=PARAMS['epsilon'], \
                                          wd=PARAMS['wd'], momentum=PARAMS['momentum'], x_id = experiment.id)
  # exp=experiment,
      train_object.get_dataset()
      # train_object.show_summary()
      # Loads the dataset according to the batch size and num_workers
      train_object.get_dataloader()
      # training
      train_object.train()
  except:
    raise
  finally:
    neptune.stop()

https://ui.neptune.ai/caioviturino/IJR2020/e/SAN-364
Experiment ID = SAN-364
 Net= faster_rcnn_resnet50_v1b_voc
Network... OK
Dataloader... OK
Train start
Start training from [Epoch 0]


/usr/local/lib/python3.6/dist-packages/mxnet/gluon/parameter.py:703: UserWarning: Constant parameter "fasterrcnn1_rpn0_rpnanchorgenerator0_anchor_" does not support grad_req other than "null", and new value "write" is ignored.
  'is ignored.'.format(self.name, req))


[Epoch 0][Batch 0], Speed: 0.719 samples/sec, RPN_Conf=0.686,RPN_SmoothL1=0.028,RCNN_CrossEntropy=2.974,RCNN_SmoothL1=0.019,RPNAcc=0.531,RPNL1Loss=0.596,RCNNAcc=0.023,RCNNL1Loss=0.273
[Epoch 0][Batch 1], Speed: 2.313 samples/sec, RPN_Conf=0.515,RPN_SmoothL1=0.432,RCNN_CrossEntropy=3.553,RCNN_SmoothL1=0.837,RPNAcc=0.738,RPNL1Loss=8.846,RCNNAcc=0.430,RCNNL1Loss=7.139
[Epoch 0][Batch 2], Speed: 6.375 samples/sec, RPN_Conf=0.455,RPN_SmoothL1=0.396,RCNN_CrossEntropy=2.842,RCNN_SmoothL1=0.566,RPNAcc=0.780,RPNL1Loss=7.794,RCNNAcc=0.596,RCNNL1Loss=5.575
[Epoch 0][Batch 3], Speed: 3.666 samples/sec, RPN_Conf=0.407,RPN_SmoothL1=0.459,RCNN_CrossEntropy=2.740,RCNN_SmoothL1=0.485,RPNAcc=0.825,RPNL1Loss=9.596,RCNNAcc=0.680,RCNNL1Loss=5.172
[Epoch 0][Batch 4], Speed: 3.562 samples/sec, RPN_Conf=0.375,RPN_SmoothL1=0.434,RCNN_CrossEntropy=2.291,RCNN_SmoothL1=0.399,RPNAcc=0.846,RPNL1Loss=8.287,RCNNAcc=0.730,RCNNL1Loss=4.476
[Epoch 0][Batch 5], Speed: 3.630 samples/sec, RPN_Conf=0.346,RPN_SmoothL1=0.388,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:249: RuntimeWarning: invalid value encountered in long_scalars
Invalid metric value: nan for channel rec_by_class_val_gear_box. Metrics with nan or +/-inf values will not be sent to server


[Epoch 0] Validation: 
bar_clamp=0.47355729010012604
gear_box=0.4785063220916743
vase=0.5760727633927589
part_1=0.7570090278865838
part_3=0.6061773343097172
nozzle=0.31224283451184714
pawn=0.30958504199123427
turbine_housing=0.4962513144186121
mAP=0.5011752410878192
drive/My Drive/UFBA/Doutorado/Ssd_test/checkpoints
drive/My Drive/UFBA/Doutorado/Ssd_test/checkpoints/faster/SAN-364/faster_rcnn_resnet50_v1b_voc_BE_0000_map_0.5012.params
Best map:  0.5011752410878192
[Epoch 1][Batch 0], Speed: 1.463 samples/sec, RPN_Conf=0.012,RPN_SmoothL1=0.005,RCNN_CrossEntropy=0.416,RCNN_SmoothL1=0.123,RPNAcc=0.984,RPNL1Loss=0.378,RCNNAcc=0.778,RCNNL1Loss=1.570
[Epoch 1][Batch 1], Speed: 3.151 samples/sec, RPN_Conf=0.009,RPN_SmoothL1=0.007,RCNN_CrossEntropy=0.458,RCNN_SmoothL1=0.128,RPNAcc=0.984,RPNL1Loss=0.377,RCNNAcc=0.778,RCNNL1Loss=1.568
[Epoch 1][Batch 2], Speed: 5.968 samples/sec, RPN_Conf=0.009,RPN_SmoothL1=0.007,RCNN_CrossEntropy=0.451,RCNN_SmoothL1=0.127,RPNAcc=0.984,RPNL1Loss=0.377,RCNNAcc=0.

KeyboardInterrupt: ignored